In [1]:
# Quick hack to load local library code
import os

os.chdir(os.path.join(os.getcwd(), ".."))

# Load API key and secret from environment variables
from dotenv import load_dotenv
load_dotenv()

True

In [2]:
import pandas as pd
import xgboost as xgb

from numpy import argmax
from sklearn.metrics import accuracy_score, precision_recall_curve
from sklearn.model_selection import train_test_split

%matplotlib inline

In [3]:
import validmind as vm

vm.init(
  api_host = "http://localhost:3000/api/v1/tracking",
  api_key = "e22b89a6b9c2a27da47cb0a09febc001",
  api_secret = "a61be901b5596e3c528d94231e4a3c504ef0bb803d16815f8dfd6857fac03e57",
  project = "cler0k3rp00010ky6yaoyopmx"
)
  

True

In [4]:
df = pd.read_csv("/Users/juanvalidmind/Dev/github/validmind/validmind-python/notebooks/datasets/lending_club_loan_rates.csv", sep='\t')

In [5]:
df.head()

,Unnamed: 0,loan_rate_A,loan_rate_B,loan_rate_C,loan_rate_D,FEDFUNDS,diff1_loan_rate_A,diff1_loan_rate_B,diff1_loan_rate_C,diff1_loan_rate_D,diff1_FEDFUNDS,diff2_FEDFUNDS
0,2007-08-01,7.766667,9.497692,10.947500,12.267000,5.02,0.060000,0.134359,0.207500,-0.467444,-0.24,-0.25
1,2007-09-01,7.841429,9.276667,10.829167,12.436667,4.94,0.074762,-0.221026,-0.118333,0.169667,-0.08,0.16
2,2007-10-01,7.830000,9.433333,10.825926,12.737368,4.76,-0.011429,0.156667,-0.003241,0.300702,-0.18,-0.10
3,2007-11-01,7.779091,9.467778,10.967037,12.609444,4.49,-0.050909,0.034444,0.141111,-0.127924,-0.27,-0.09
4,2007-12-01,7.695833,9.387500,10.805000,12.478889,4.24,-0.083258,-0.080278,-0.162037,-0.130556,-0.25,0.02


In [6]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 137 entries, 0 to 136
Data columns (total 12 columns):
 #   Column             Non-Null Count  Dtype  
---  ------             --------------  -----  
 0   Unnamed: 0         137 non-null    object 
 1   loan_rate_A        137 non-null    float64
 2   loan_rate_B        137 non-null    float64
 3   loan_rate_C        137 non-null    float64
 4   loan_rate_D        137 non-null    float64
 5   FEDFUNDS           137 non-null    float64
 6   diff1_loan_rate_A  137 non-null    float64
 7   diff1_loan_rate_B  137 non-null    float64
 8   diff1_loan_rate_C  137 non-null    float64
 9   diff1_loan_rate_D  137 non-null    float64
 10  diff1_FEDFUNDS     137 non-null    float64
 11  diff2_FEDFUNDS     137 non-null    float64
dtypes: float64(11), object(1)
memory usage: 13.0+ KB


In [7]:
df = df.rename(columns={'Unnamed: 0': 'Date'})
df = df.set_index(pd.to_datetime(df['Date']))
df.drop(["Date"], axis=1, inplace=True)
df

,loan_rate_A,loan_rate_B,loan_rate_C,loan_rate_D,FEDFUNDS,diff1_loan_rate_A,diff1_loan_rate_B,diff1_loan_rate_C,diff1_loan_rate_D,diff1_FEDFUNDS,diff2_FEDFUNDS
Date,,,,,,,,,,,
2007-08-01,7.766667,9.497692,10.947500,12.267000,5.02,0.060000,0.134359,0.207500,-0.467444,-0.24,-0.25
2007-09-01,7.841429,9.276667,10.829167,12.436667,4.94,0.074762,-0.221026,-0.118333,0.169667,-0.08,0.16
2007-10-01,7.830000,9.433333,10.825926,12.737368,4.76,-0.011429,0.156667,-0.003241,0.300702,-0.18,-0.10
2007-11-01,7.779091,9.467778,10.967037,12.609444,4.49,-0.050909,0.034444,0.141111,-0.127924,-0.27,-0.09
2007-12-01,7.695833,9.387500,10.805000,12.478889,4.24,-0.083258,-0.080278,-0.162037,-0.130556,-0.25,0.02
...,...,...,...,...,...,...,...,...,...,...,...
2018-08-01,7.218997,11.161286,15.142618,19.857603,1.91,0.052118,0.045181,0.056796,0.088167,0.00,-0.09
2018-09-01,7.201281,11.191918,15.139769,19.748459,1.95,-0.017716,0.030632,-0.002849,-0.109144,0.04,0.04
2018-10-01,7.228498,11.208418,15.129105,19.792163,2.19,0.027218,0.016500,-0.010665,0.043704,0.24,0.20


In [8]:
vm_dataset = vm.init_dataset(
    dataset=df,
    target_column="loan_rate_A",    
)

Pandas dataset detected. Initializing VM Dataset instance...
Inferring dataset types...


In [9]:
vm.test_plans.list_plans()

ID,Name,Description
sklearn_classifier_metrics,SKLearnClassifierMetrics,Test plan for sklearn classifier metrics
sklearn_classifier_validation,SKLearnClassifierPerformance,Test plan for sklearn classifier models
sklearn_classifier,SKLearnClassifier,Test plan for sklearn classifier models that includes both metrics and validation tests
tabular_dataset,TabularDataset,Test plan for generic tabular datasets
tabular_dataset_description,TabularDatasetDescription,Test plan to extract metadata and descriptive statistics from a tabular dataset
tabular_data_quality,TabularDataQuality,Test plan for data quality on tabular datasets
normality_test_plan,NormalityTestPlan,Test plan to perform normality tests.
autocorrelation_test_plan,AutocorrelationTestPlan,Test plan to perform autocorrelation tests.
seasonality_test_plan,SesonalityTestPlan,Test plan to perform seasonality tests.
unit_root_test_plan,UnitRootTestPlan,Test plan to perform unit root tests.


In [10]:
test_size = 30
train_ds = df[:-test_size]
test_ds = df[-test_size:]

In [11]:
train_ds.head()

,loan_rate_A,loan_rate_B,loan_rate_C,loan_rate_D,FEDFUNDS,diff1_loan_rate_A,diff1_loan_rate_B,diff1_loan_rate_C,diff1_loan_rate_D,diff1_FEDFUNDS,diff2_FEDFUNDS
Date,,,,,,,,,,,
2007-08-01,7.766667,9.497692,10.947500,12.267000,5.02,0.060000,0.134359,0.207500,-0.467444,-0.24,-0.25
2007-09-01,7.841429,9.276667,10.829167,12.436667,4.94,0.074762,-0.221026,-0.118333,0.169667,-0.08,0.16
2007-10-01,7.830000,9.433333,10.825926,12.737368,4.76,-0.011429,0.156667,-0.003241,0.300702,-0.18,-0.10
2007-11-01,7.779091,9.467778,10.967037,12.609444,4.49,-0.050909,0.034444,0.141111,-0.127924,-0.27,-0.09
2007-12-01,7.695833,9.387500,10.805000,12.478889,4.24,-0.083258,-0.080278,-0.162037,-0.130556,-0.25,0.02


In [12]:
test_ds.head()

,loan_rate_A,loan_rate_B,loan_rate_C,loan_rate_D,FEDFUNDS,diff1_loan_rate_A,diff1_loan_rate_B,diff1_loan_rate_C,diff1_loan_rate_D,diff1_FEDFUNDS,diff2_FEDFUNDS
Date,,,,,,,,,,,
2016-07-01,7.409694,10.481218,14.012341,18.673089,0.39,0.287472,0.307986,0.231464,0.388211,0.01,0.00
2016-08-01,7.415676,10.430687,14.018968,18.725052,0.40,0.005982,-0.050531,0.006627,0.051963,0.01,0.00
2016-09-01,7.309269,10.420683,13.995526,18.801644,0.40,-0.106407,-0.010004,-0.023442,0.076592,0.00,-0.01
2016-10-01,7.155385,10.461157,14.069014,18.808550,0.40,-0.153885,0.040474,0.073488,0.006906,0.00,0.00
2016-11-01,7.026322,10.626165,14.117571,18.773467,0.41,-0.129063,0.165008,0.048557,-0.035083,0.01,0.01


In [13]:
vm_train_ds = vm.init_dataset(dataset=train_ds, type="generic", target_column="loan_rate_A")
vm_test_ds = vm.init_dataset(dataset=test_ds, type="generic", target_column="loan_rate_A")

Pandas dataset detected. Initializing VM Dataset instance...
Inferring dataset types...
Pandas dataset detected. Initializing VM Dataset instance...
Inferring dataset types...


In [14]:
vm.run_test_plan("seasonality_test_plan", train_ds=vm_train_ds, test_ds=vm_test_ds)

Logging result: TestPlanMetricResult:   0%|          | 0/1 [00:00<?, ?it/s]                                          

Could not log figure to ValidMind API


Exception: <!doctype html>
<html lang=en>
  <head>
    <title>botocore.exceptions.NoCredentialsError: Unable to locate credentials
 // Werkzeug Debugger</title>
    <link rel="stylesheet" href="?__debugger__=yes&amp;cmd=resource&amp;f=style.css">
    <link rel="shortcut icon"
        href="?__debugger__=yes&amp;cmd=resource&amp;f=console.png">
    <script src="?__debugger__=yes&amp;cmd=resource&amp;f=debugger.js"></script>
    <script>
      var CONSOLE_MODE = false,
          EVALEX = true,
          EVALEX_TRUSTED = false,
          SECRET = "5yYXTpmOOAqXzy0EhTN2";
    </script>
  </head>
  <body style="background-color: #fff">
    <div class="debugger">
<h1>NoCredentialsError</h1>
<div class="detail">
  <p class="errormsg">botocore.exceptions.NoCredentialsError: Unable to locate credentials
</p>
</div>
<h2 class="traceback">Traceback <em>(most recent call last)</em></h2>
<div class="traceback">
  <h3></h3>
  <ul><li><div class="frame" id="frame-6163219504">
  <h4>File <cite class="f

In [17]:
vm.run_test_plan("stationarity_test_plan", train_ds=vm_train_ds, test_ds=vm_test_ds)

Running Metric: kpss:  20%|██        | 1/5 [00:00<00:00, 41.61it/s]                                                   The test statistic is outside of the range of p-values available in the
look-up table. The actual p-value is greater than the p-value returned.

The test statistic is outside of the range of p-values available in the
look-up table. The actual p-value is smaller than the p-value returned.

The test statistic is outside of the range of p-values available in the
look-up table. The actual p-value is smaller than the p-value returned.

The test statistic is outside of the range of p-values available in the
look-up table. The actual p-value is smaller than the p-value returned.

The test statistic is outside of the range of p-values available in the
look-up table. The actual p-value is greater than the p-value returned.

The test statistic is outside of the range of p-values available in the
look-up table. The actual p-value is greater than the p-value returned.

The test sta

In [ ]:
vm.run_test_plan("timeseries_test_plan", test_ds=vm_test_ds, train_ds=vm_train_ds)